This program uses Pyspark SQL to extract customer, branch and card data from the provided json file.

In [30]:
# Import libraries 
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import *

# 1. Functional Requirements - Load Credit Card Database (SQL)

<b>Data Extraction and Transformation with Python and PySpark. </b><br>
For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.
1. CDW_SAPP_BRANCH.JSON <br>
2. CDW_SAPP_CREDITCARD.JSON <br>
3. CDW_SAPP_CUSTOMER.JSON <br>
Note: Data Engineers will be required to transform the data based on the requirements found in the Mapping Document.
Hint: [You can use PYSQL “select statement query” or simple Pyspark RDD].

In [31]:
# Application to create Dataframes from source
spark = SparkSession.builder.master('local[1]').appName('CreditCardSystems').getOrCreate() 

# Extract the JSON files branch, credit and customer into a dataframe
df_branch = spark.read.json('cdw_sapp_branch.json')  
df_credit = spark.read.json('cdw_sapp_credit.json') 
df_customer = spark.read.json('cdw_sapp_customer.json')


In [32]:
df_branch.dtypes

[('BRANCH_CITY', 'string'),
 ('BRANCH_CODE', 'bigint'),
 ('BRANCH_NAME', 'string'),
 ('BRANCH_PHONE', 'string'),
 ('BRANCH_STATE', 'string'),
 ('BRANCH_STREET', 'string'),
 ('BRANCH_ZIP', 'bigint'),
 ('LAST_UPDATED', 'string')]

In [33]:
# Adjust branch according the mapping document

# Change datatype to integer
df_branch = df_branch.withColumn("BRANCH_CODE", df_branch["BRANCH_CODE"].cast(IntegerType()))
df_branch = df_branch.withColumn("BRANCH_ZIP", df_branch["BRANCH_ZIP"].cast(IntegerType()))

# Change datatype to varchar - COnvert it on the sql side
df_branch = df_branch.withColumn("BRANCH_CITY", df_branch["BRANCH_CITY"].cast(VarcharType()))


NameError: name 'VarcharType' is not defined

In [34]:
df_branch.dtypes

[('BRANCH_CITY', 'string'),
 ('BRANCH_CODE', 'int'),
 ('BRANCH_NAME', 'string'),
 ('BRANCH_PHONE', 'string'),
 ('BRANCH_STATE', 'string'),
 ('BRANCH_STREET', 'string'),
 ('BRANCH_ZIP', 'int'),
 ('LAST_UPDATED', 'string')]

In [24]:
df_credit.show(4)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|
|        114|4210653349028689|123459988| 19|    4|             4|   Entertainment|            59.73|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 4 rows



In [16]:
df_customer.show(5)

+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States|WDunham@example.co

<b>Data loading into Database </b><br>
Once PySpark reads data from JSON files, and then utilizes Python, PySpark, and Python modules to load data into RDBMS(SQL), perform
the following: <br>
a) Create a Database in SQL(MariaDB), named “creditcard_capstone.” <br>
b) Create a Python and Pyspark Program to load/write the “Credit Card System Data” into RDBMS(creditcard_capstone). <br>
Tables should be created by the following names in RDBMS: <br>
CDW_SAPP_BRANCH <br>
CDW_SAPP_CREDIT_CARD <br>
CDW_SAPP_CUSTOMER <br>

In [18]:
# Create the table CDW_SAPP_BRANCH 
df_branch.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_BRANCH") \
.option("user", "root") \
.option("password", "a") \
.save()

In [19]:
# Create the table CDW_SAPP_CREDIT_CARD 
df_credit.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CREDIT_CARD") \
.option("user", "root") \
.option("password", "a") \
.save()

In [20]:
# Create the table CDW_SAPP_CUSTOMER 
df_customer.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.CDW_SAPP_CUSTOMER") \
.option("user", "root") \
.option("password", "a") \
.save()

# 2. Functional Requirements - Application Front-End
Once data is loaded into the database, we need a front-end (console) to see/display data. For that, create a console-based Python program to satisfy System Requirements 2 (2.1 and 2.2).

<b> Transaction Details Module </b><br>
1) Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in
descending order. <br>
2) Used to display the number and total values of transactions for a given type.<br>
3) Used to display the number and total values of transactions for branches in a given state.<br>

# 2.1 Order by day in descending order.

In [ ]:
# INPUT 
Month = 0 # Holds the input value for month     | 8
Year = 0  # Holds the input value for year      | 2018
Zipcode = 0 # Holds the input value for zipcode | 39120

# EXPECTED OUTPUT 
# CUST_ZIP | MONTH | YEAR | DAY | TRANSACTION_VALUE all three transactions 
# 39120    | 8     | 2018 | 14  |  29.15
# 39120    | 8     | 2018 | 5   |  36.3

# Use cdw_app_credit_card table to get TRANSACTION_VALUE, DAY, MONTH AND YEAR
# Use cdw_app_customer to get CUST_ZIP 







<b>Customer Details </b><br>
1) Used to check the existing account details of a customer.<br>
2) Used to modify the existing account details of a customer.<br>
3) Used to generate a monthly bill for a credit card number for a given month and year. <br>
4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order. <br>

# 3 - Functional Requirements - Data analysis and Visualization

After data is loaded into the database, users can make changes from the front end, and they can also view data from the front end. Now, the business analyst team wants to analyze and visualize the data according to the below requirements.

<b> Data Analysis and Visualization </b> <br>
1) Find and plot which transaction type has a high rate of transactions. <br>
2) Find and plot which state has a high number of customers. <br>
3) Find and plot the sum of all transactions for each customer, and which customer has the highest transaction amount.hint(use CUST_SSN).



# 4. Functional Requirements - LOAN Application Dataset

1. Create a Python program to GET (consume) data from the above API endpoint for the loan application dataset. <br>
2. Find the status code of the above API endpoint. <br>
3. Once Python reads data from the API, utilize PySpark to load data into RDBMS(SQL). The table name should be CDW-SAPP_loan_application in the database.

# 5 - Functional Requirements - Data Analysis and Visualization for Loan Application

1. Find and plot the percentage of applications approved for self-employed applicants. <br>
2. Find the percentage of rejection for married male applicants. <br>
3. Find and plot the top three months with the largest transaction data.<br>
4. Find and plot which branch processed the highest total dollar value of healthcare transactions.